# 📊 Analyse Économétrique Interactive
## Impact du CO2 sur le Dow Jones

Ce notebook permet d'exécuter l'analyse de manière interactive et d'explorer les résultats.

### 1. Imports et Configuration

In [ ]:
import sys
import os

# Ajouter le dossier src au path
sys.path.insert(0, os.path.join(os.getcwd(), 'src'))

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from data_collector import DataCollector
from statistical_analysis import StatisticalAnalyzer
from visualizations import Visualizer

# Configuration de l'affichage
%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 4)

print("✅ Imports réussis")

### 2. Collecte des Données

In [ ]:
# Collecter les données
collector = DataCollector(start_date="2015-01-01")
data = collector.collect_all_data()

print(f"\n📊 {len(data)} observations collectées")
print(f"Période: {data['Date'].min()} à {data['Date'].max()}")

# Afficher les premières lignes
data.head()

In [ ]:
# Informations sur le dataset
data.info()

### 3. Statistiques Descriptives

In [ ]:
# Initialiser l'analyseur
analyzer = StatisticalAnalyzer(data)

# Calculer les statistiques
stats = analyzer.compute_descriptive_stats()

# Afficher sous forme de DataFrame
stats_df = pd.DataFrame(stats)
stats_df

### 4. Visualisations Exploratoires

In [ ]:
# Évolution temporelle
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Dow Jones
axes[0].plot(data['Date'], data['DJI_Close'], linewidth=1.5)
axes[0].set_title('Évolution du Dow Jones', fontsize=14, fontweight='bold')
axes[0].set_ylabel('DJI Close')
axes[0].grid(True, alpha=0.3)

# CO2
axes[1].plot(data['Date'], data['CO2_Level'], linewidth=1.5, color='red')
axes[1].set_title('Évolution du CO2', fontsize=14, fontweight='bold')
axes[1].set_ylabel('CO2 (ppm)')
axes[1].set_xlabel('Date')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 5. Corrélation

In [ ]:
# Calculer la corrélation
correlation = analyzer.compute_correlation()
print(f"Corrélation DJI-CO2: {correlation:.4f}")

# Matrice de corrélation
corr_matrix = data[['DJI_Close', 'CO2_Level']].corr()

# Heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, fmt='.4f', cmap='coolwarm', 
            center=0, square=True, linewidths=2)
plt.title('Matrice de Corrélation', fontsize=14, fontweight='bold')
plt.show()

### 6. Régression Linéaire

In [ ]:
# Effectuer la régression OLS
model = analyzer.run_ols_regression()

# Afficher le résumé
print(model.summary())

In [ ]:
# Visualiser la régression
plt.figure(figsize=(12, 8))
sns.regplot(x='CO2_Level', y='DJI_Close', data=data,
           scatter_kws={'alpha': 0.4, 's': 30},
           line_kws={'color': 'red', 'linewidth': 2.5})
plt.title(f'Régression Linéaire (R² = {model.rsquared:.4f})', 
         fontsize=14, fontweight='bold')
plt.xlabel('CO2 (ppm)', fontsize=12)
plt.ylabel('Dow Jones', fontsize=12)
plt.grid(True, alpha=0.3)
plt.show()

### 7. Tests Statistiques

In [ ]:
# Test de normalité
norm_results = analyzer.test_normality()
print("Test de Normalité (Shapiro-Wilk):")
print(pd.DataFrame(norm_results))

In [ ]:
# Test de stationnarité
stat_results = analyzer.test_stationarity()
print("Test de Stationnarité (ADF):")
print(pd.DataFrame(stat_results))

In [ ]:
# Tests d'hétéroscédasticité
het_results = analyzer.test_heteroscedasticity()
print("Tests d'Hétéroscédasticité:")
print(f"Breusch-Pagan p-value: {het_results['breusch_pagan']['lm_p_value']:.4e}")
print(f"White p-value: {het_results['white']['lm_p_value']:.4e}")
print(f"Hétéroscédasticité détectée: {het_results['has_heteroscedasticity']}")

In [ ]:
# Tests d'autocorrélation
auto_results = analyzer.test_autocorrelation()
print("Tests d'Autocorrélation:")
print(f"Durbin-Watson: {auto_results['durbin_watson']['statistic']:.4f}")
print(f"Breusch-Godfrey p-value: {auto_results['breusch_godfrey']['lm_p_value']:.4e}")
print(f"Autocorrélation détectée: {auto_results['has_autocorrelation']}")

### 8. Analyse des Résidus

In [ ]:
# Créer le visualiseur
viz = Visualizer(data, model)

# Analyser les résidus (génère l'image)
viz.plot_residuals_analysis()

# Afficher l'image
from IPython.display import Image
Image('output/residuals_analysis.png')

### 9. Conclusion

Les résultats de cette analyse montrent que :
- La corrélation entre CO2 et Dow Jones est de **{correlation:.4f}**
- Le modèle explique **{model.rsquared*100:.2f}%** de la variance
- Le coefficient de régression est **statistiquement significatif** (p < 0.05)
